In [2]:
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
save_root = root + "/Step_1_data/Dataset_netrin_ST/"
import sys
sys.path.append(code_loc ) 

In [3]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part 
vocab_loc = code_loc +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'Ground_Truth' # the label to be predicted

In [4]:

csv_path = root+"/Step_0_data/netrin_spatial_expression_data_filtered_unnormalised.csv" 
data_path = root +  "/Step_0_data/2024_03_13_EMT_netrin_ST.h5ad"


In [5]:
import pandas as pd

df = pd.read_csv(csv_path)

In [6]:
df

,Unnamed: 0,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AGRN,C1orf159,TNFRSF18,...,DKC1,MPP1,FUNDC2,CMC4,MTCP1,VBP1,CLIC2,TMLHE,SPRY3,VAMP7
0,AAACTGCTGGCTCCAA-1_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,AAAGGGATGTAGCAAG-1_0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AACCTCGCTTTAGCCC-1_0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,AACGCGGTCTCCAGCC-1_0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,AACGTGCGAAAGTCTC-1_0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,TTGTTAGCAAATTCGA-1_3,3.0,5.0,0.0,1.0,11.0,0.0,13.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
2619,TTGTTCAGTGTGCTAC-1_3,0.0,3.0,0.0,0.0,20.0,1.0,13.0,0.0,0.0,...,11.0,0.0,1.0,6.0,0.0,6.0,0.0,3.0,0.0,3.0
2620,TTGTTGTGTGTCAAGA-1_3,2.0,4.0,0.0,0.0,6.0,4.0,10.0,0.0,0.0,...,3.0,0.0,2.0,6.0,0.0,4.0,0.0,1.0,2.0,1.0
2621,TTGTTTCATTAGTCTA-1_3,4.0,5.0,2.0,0.0,4.0,1.0,6.0,0.0,0.0,...,2.0,0.0,1.0,2.0,0.0,4.0,0.0,0.0,0.0,1.0


In [7]:
sample_id = df.iloc[:,0].values
gene_id = df.columns[1:].values
data_x = df.iloc[:,1:].values

print(data_x.shape, len(sample_id), len(gene_id))

import numpy as np
# confirm nan and inf in data_x
print(np.isnan(data_x).any(), np.isinf(data_x).any())
# set nan to 0
data_x = np.nan_to_num(data_x)

# normalize the data to 0-100
# 找到数据的最小值和最大值
min_val = np.min(data_x)
max_val = np.max(data_x)
print(min_val, max_val )
# 将数据归一化到 0 到 100 的范围
normalized_data_x = 100 * (data_x - min_val) / (max_val - min_val)
print(normalized_data_x.shape)
# create the adata matrix
import anndata
adata = anndata.AnnData(X = normalized_data_x, obs = pd.DataFrame(index = sample_id), var = pd.DataFrame(index = gene_id))
adata.obs["Ground_Truth"] = np.random.choice([0,1], adata.shape[0])
print(adata, adata.obs, adata.var, adata.X)

(2623, 12134) 2623 12134
False False
0.0 1698.0
(2623, 12134)
AnnData object with n_obs × n_vars = 2623 × 12134
    obs: 'Ground_Truth'                       Ground_Truth
AAACTGCTGGCTCCAA-1_0             0
AAAGGGATGTAGCAAG-1_0             1
AACCTCGCTTTAGCCC-1_0             1
AACGCGGTCTCCAGCC-1_0             1
AACGTGCGAAAGTCTC-1_0             1
...                            ...
TTGTTAGCAAATTCGA-1_3             0
TTGTTCAGTGTGCTAC-1_3             0
TTGTTGTGTGTCAAGA-1_3             1
TTGTTTCATTAGTCTA-1_3             1
TTGTTTGTGTAAATTC-1_3             0

[2623 rows x 1 columns] Empty DataFrame
Columns: []
Index: [SAMD11, NOC2L, KLHL17, PLEKHN1, HES4, ISG15, AGRN, C1orf159, TNFRSF18, TNFRSF4, SDF4, B3GALT6, UBE2J2, SCNN1D, ACAP3, PUSL1, INTS11, CPTP, DVL1, MXRA8, AURKAIP1, CCNL2, ANKRD65, VWA1, ATAD3C, TMEM240, SSU72, FNDC10, MIB2, MMP23B, CDK11B, SLC35E2B, SLC35E2A, NADK, GNB1, CALML6, PRKCZ, FAAP20, SKI, MORN1, RER1, PEX10, PLCH2, PANK4, TNFRSF14, PRXL2B, ARHGEF16, MEGF6, TPRG1L, WRAP73, 

In [8]:
#adata.write(data_path)

In [9]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np
adata = sc.read(data_path)

print(adata)
print(adata.obs.index)
print(adata.var.index)
print(f" negative values: {np.sum(adata.X<0)}")

AnnData object with n_obs × n_vars = 2623 × 12134
    obs: 'Ground_Truth'
Index(['AAACTGCTGGCTCCAA-1_0', 'AAAGGGATGTAGCAAG-1_0', 'AACCTCGCTTTAGCCC-1_0',
       'AACGCGGTCTCCAGCC-1_0', 'AACGTGCGAAAGTCTC-1_0', 'AACTCCAGAGCGTGTT-1_0',
       'AAGTGTTTGGAGACGG-1_0', 'AAGTTCACTCCAAGCT-1_0', 'AATGCAACCGGGTACC-1_0',
       'ACAAGGAAATCCGCCC-1_0',
       ...
       'TTGTAATCCGTACTCG-1_3', 'TTGTATCACACAGAAT-1_3', 'TTGTCGTTCAGTTACC-1_3',
       'TTGTGGCCCTGACAGT-1_3', 'TTGTGGTAGGAGGGAT-1_3', 'TTGTTAGCAAATTCGA-1_3',
       'TTGTTCAGTGTGCTAC-1_3', 'TTGTTGTGTGTCAAGA-1_3', 'TTGTTTCATTAGTCTA-1_3',
       'TTGTTTGTGTAAATTC-1_3'],
      dtype='object', length=2623)
Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'HES4', 'ISG15', 'AGRN',
       'C1orf159', 'TNFRSF18', 'TNFRSF4',
       ...
       'DKC1', 'MPP1', 'FUNDC2', 'CMC4', 'MTCP1', 'VBP1', 'CLIC2', 'TMLHE',
       'SPRY3', 'VAMP7'],
      dtype='object', length=12134)
 negative values: 0


In [10]:
# adata.X 有多少nan
import numpy as np
from scipy.sparse import issparse

# 假设 adata.X 是你的稀疏矩阵
# 首先，检查它是否为稀疏矩阵
if issparse(adata.X):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(adata.X.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(adata.X).sum()

print(f"NaN values in adata.X: {nan_count} with shape {adata.X.shape}")


NaN values in adata.X: 0 with shape (2623, 12134)


In [11]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT 
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_Truth",
                            vocab_loc=code_loc + "/Experiment/support_data/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=None,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc=code_loc +"/Experiment/support_data/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=None,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='Ground_Truth', vocab_loc='/home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=200, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='Ground_Truth', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=None, random_state=2023, shuffle=True, sort_seq_batch=False)


In [12]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para=dataset_para)
# init vocab
data_reader.init_vocab()

# load data
data_reader.load_adata(loc = data_path,translate=True)

## preprocess
data_reader.preprocess()

/home/shi/anaconda3/envs/scLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json
scLLM - INFO - Load data from anndata object.
scLLM - DEBUG - In original adata with gene 12134
scLLM - DEBUG - In original adata with gene 12134
scLLM - DEBUG - processing 0/16906
scLLM - DEBUG - processing 2000/16906
scLLM - DEBUG - processing 4000/16906
scLLM - DEBUG - processing 6000/16906
scLLM - DEBUG - processing 8000/16906
scLLM - DEBUG - processing 10000/16906
scLLM - DEBUG - processing 12000/16906
scLLM - DEBUG - processing 14000/16906
scLLM - DEBUG - processing 16000/16906
scLLM - INFO - create anndata 

In [13]:
print(data_reader.adata, data_reader.adata.X.shape)

AnnData object with n_obs × n_vars = 2532 × 16906
    obs: 'Ground_Truth', 'n_counts'
    uns: 'log1p'
    layers: 'X_normed', 'X_log1p' (2532, 16906)


In [14]:
# get layer X_log1p
data = data_reader.adata.layers["X_log1p"] 
from scipy.sparse import coo_matrix, csr_matrix
# set nan to 0
if isinstance(data, csr_matrix):
    # 使用 CSR 格式的稀疏矩阵可以直接通过 data.data 访问非零元素
    data.data[np.isnan(data.data)] = 0
    # 注意: 这里没有改变矩阵的稀疏结构，只是将非零元素中的 NaN 值替换为 0
    data.data[np.isinf(data.data)] = data.data.max()
else:
    # 如果 data 不是稀疏矩阵，可以直接使用 numpy 的方法
    data[np.isnan(data)] = 0
    data[np.isinf(data)] = data.max()

if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(data.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(data).sum()

print(f"NaN values in adata.X: {nan_count}")
data_reader.adata.layers["X_log1p"]  = data 

NaN values in adata.X: 0


In [15]:
# data <0 的个数
if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.sum(data.toarray()<0)
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.sum(data<0)

print(f"Negative values in adata.X: {nan_count}")

Negative values in adata.X: 0


In [16]:
print(data.shape)

(2532, 16906)


In [17]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
label_dict = {'fake_lable_0':0,'fake_lable_1':1}

[1 0]


In [18]:
# set fake label
#data_reader.adata.obs[dataset_para.label_key]
revert_dict = {v:k for k,v in label_dict.items()}
# map to str label
data_reader.adata.obs[dataset_para.label_key] = data_reader.adata.obs[dataset_para.label_key].map(revert_dict)

In [19]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'fake_lable_1': 0, 'fake_lable_0': 1}
scLLM - INFO - Discritize label Ground_Truth in obs_names


trainset size:  2532
no valset
{'fake_lable_0': 0, 'fake_lable_1': 1}
weights:  None


In [20]:
trainset.sample_id = data_reader.adata.obs.index.to_list()

In [21]:
import dill
# 为trainset 添加其他labels

#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
loc = save_root + target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,label_dict],f)

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import issparse
# 假设adata和raw_adata是已经加载的Anndata对象

# 定义一个函数来计算稀疏矩阵每行的哈希值
def hash_sparse_row(row):
    # 将稀疏矩阵行转换为密集格式然后计算哈希
    return hash(tuple(row.toarray()[0]))

data1 = trainset.data #adata.X
data2 = data_reader.adata.layers["X_log1p"] #raw_adata.X
data2_source = data_reader.adata
# 检查adata.X是否为稀疏矩阵，然后相应地计算哈希值
if issparse(data1):
    adata_hashes = [hash_sparse_row(data1[row,:]) for row in range(data1.shape[0])]
else:
    adata_hashes = [hash(tuple(row)) for row in adata.X.toarray()]

# 同样的处理对raw_adata.X进行
if issparse(data2):
    raw_adata_hashes = [hash_sparse_row(data2[row,:]) for row in range(data2.shape[0])]
else:
    raw_adata_hashes = [hash(tuple(row)) for row in data2.toarray()]

# 创建从raw_adata的行哈希值到其obs.index的映射
hash_to_index_map = {hash_: index for hash_, index in zip(raw_adata_hashes, data2_source.obs.index)}


# 创建一个列表来存储对应的raw_adata.obs.index
mapped_indices = []

# 对于adata中的每一行，找到对应的raw_adata.obs.index
for i,hash_ in enumerate(adata_hashes):
    mapped_index = hash_to_index_map.get(hash_, None)
    if mapped_index is not None:
        mapped_indices.append(mapped_index)
    else:
        # 如果找不到对应，可能需要处理这种情况
        raise ValueError(f"No matching index {i} found for hash: {str(hash_)}" )

# 将mapped_indices转换为Pandas Series对象，以便更方便地处理
mapped_series = pd.Series(mapped_indices, index=adata.obs.index)

# 打印或者返回映射结果
print(mapped_series)
